In [10]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
from IPython.display import display
def display_matrix(m):
    display(sympy.Matrix(m))
import sympy

sympy.init_printing()

In [11]:
S0 = 100
T = 1
mu = 0.05
sigma = 0.2
lam = 1
I = 100
m = 0
v = 0.3
paths = 100
K = 99
df = np.exp(-mu*T)

In [12]:
def matrix(paths):
    np.random.seed(42)
    matrix = np.zeros((paths, I))
    for k in range(paths):
        X = np.zeros(I)
        S = np.zeros(I)

        X[0] = np.log(S0)
        S[0] = S0
        dt = T / I
    
        for i in range(1,I):
            Z = np.random.standard_normal()
            N = np.random.poisson(lam * dt)
            Y = np.exp(np.random.normal(m,v,N))
    
            if N == 0:
                M = 0
    
            else:
                for j in range(N):
                    M = np.sum(np.log(Y))
                    
            X[i] = X[i-1] + (mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z + M
            S[i] = np.exp(X[i])
        matrix[k] = S
    return matrix

S = matrix(paths)
S.shape

In [14]:
H = np.maximum(K - S, 0)  # intrinsic values for put option
V = np.zeros_like(H)  # value matrix
V[:, -1] = H[:, -1] # set value at maturity equal to intrinsic value

# Valuation by LS Method
for t in range(I - 2, 0, -1): #start at t = I-2 and go backwards (since we already set t = I-1 to intrinsic value)
    good_paths = H[:, t] > 0  # paths where the intrinsic value is positive
    # the regression is performed only on these paths

    # Now we create the X matrix with the new basis functions
    S_t = S[good_paths, t]  # stock prices at time t
    S_t_1 = S[:, 1]  # S1 prices
    S_t_2 = S[:, 2]  # S2 prices assuming S is a two-asset option model

    X = np.column_stack((S_t, S_t**2, S_t**3))

    # Regression: Predicting the continuation value
    Y = V[good_paths, t + 1] * df  # discounted option values at t+1
    reg = np.linalg.lstsq(X, Y, rcond=None)[0]  # perform least squares regression

    # Using the regression coefficients to predict continuation values
    C = X @ reg  # matrix multiplication to get continuation values

    exercise = np.zeros(len(good_paths), dtype=bool)  # initialize
    exercise[good_paths] = H[good_paths, t] > C  # paths where it is optimal to exercise

    V[exercise, t] = H[exercise, t]  # set V equal to H where it is optimal to exercise
    V[exercise, t + 1 :] = 0  # set future cash flows, for that path, equal to zero
    discount_path = V[:, t] == 0  # paths where we didn't exercise
    V[discount_path, t] = V[discount_path, t + 1] * df  # set V[t] in continuation region

V0 = np.mean(V[:, 1]) * df  # discounted expectation of V[t=1]
print('The value of the American put option is: ', V0)

V.shape

The value of the American put option is:  3.676123705643855
